In [1]:
import requests
import json
import pickle
import pandas as pd
import ast

In [73]:
MY_API_KEY = ""

In [11]:
##creating the forloop to compare restaurants scraped from zlatestranky and from google
from LinkGetter import LinkGetter
from Restaurant import Restaurant
links=LinkGetter()

Successfuly requested 22 pages


In [3]:
links_sample = links.links[:50]
links_sample[20]

'https://www.zlatestranky.cz/profil/H168141'

In [65]:
#This forloop was used just for inspecting how the API works, it can be removed later
#MY_API_KEY = ""

n = 1
for i in links_sample:
    rest = Restaurant(i)

    iter_name = rest.name
    iter_address = rest.address
    iter_lat = rest.coordinates["latitude"]
    iter_long = rest.coordinates["longitude"]

    iter_place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={iter_name} {iter_address}&inputtype=textquery&locationbias=circle:50@{iter_lat},{iter_long}&fields=business_status%2Cformatted_address%2Cgeometry%2Cname%2Cplace_id%2Cplus_code%2Ctype%2Copening_hours&key={MY_API_KEY}")
    iter_place_dict = json.loads(iter_place_request.text)

    print("Restaurant", n, ":", rest.name)
    print(rest.review_count,"ratings,   ", rest.ratings, "stars")
    print(rest.opening_hours_span)
    print(rest.district)
    n += 1
    print("---Google info---") # Here, a condition will need to be added, in case of zero candidates return None

    if iter_place_dict["candidates"] == []:
        print("No place found")
    else:
        print("## Find Place information ##")
        print(iter_place_dict["candidates"][0]["name"])
        print(iter_place_dict["candidates"][0]["formatted_address"])
        #print(iter_place_dict["candidates"][0]["opening_hours"])

        print("## Place Details information ##")

        iter_id = iter_place_dict["candidates"][0]["place_id"]
        iter_detail_request = requests.get(f"https://maps.googleapis.com/maps/api/place/details/json?place_id={iter_id}&fields=formatted_address%2Cname%2Crating%2Cformatted_phone_number&key={MY_API_KEY}")
        iter_detail_dict = json.loads(iter_detail_request.text)
        print(iter_detail_dict['result']['name'])
        print(iter_detail_dict['result']['formatted_address'])

    print("## Find place by MOBILE Phone ##")
    iter_phones = []
    for p in rest.phones:
        iter_phones.append(rest.phones[p])
    iter_main_phone=iter_phones[0]

    print("Restaurant's phones:", rest.phones)
    print("Restaurant's main phone:", iter_main_phone)

    iter_phone_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={iter_main_phone}&inputtype=textquery&locationbias=circle:50@{iter_lat},{iter_long}&fields=business_status%2Cformatted_address%2Cgeometry%2Cname%2Cplace_id%2Cplus_code%2Ctype%2Copening_hours&key={MY_API_KEY}")
    iter_phone_dict = json.loads(iter_phone_request.text)

    if iter_phone_dict["candidates"] == []:
        print("No place found")
    else:
        print(iter_phone_dict["candidates"][0]["name"])
        print(iter_phone_dict["candidates"][0]["formatted_address"])
        #print(iter_phone_dict["candidates"][0]["formatted_phone_number"])

    print("#######################")


Restaurant 1 : Repre restaurant
0 ratings,    0.0 stars
{'Po': 12.5, 'Út': 12.5, 'St': 12.5, 'Čt': 12.5, 'Pá': 12.5, 'So': 11.0, 'Ne': None}
Praha 1
---Google info---
## Find Place information ##
Repre restaurant
Nekázanka 4, 110 00 Nové Město, Česko
## Place Details information ##
Repre Restaurant
857, Nekázanka 4, 110 00 Praha 1-Nové Město, Czechia
## Find place by MOBILE Phone ##
Restaurant's phones: {'Restaurace Praha 1': '+420 222 211 451'}
Restaurant's main phone: +420 222 211 451
Repre restaurant
Nekázanka 4, 110 00 Nové Město, Česko


KeyError: 'formatted_phone_number'

In [87]:
#First framework of function to download the API data
#Firstly retrieving place_id of each restaurant
place_ids=[]
for link in links_sample:
    restaurant=Restaurant(link)
    iter_name = restaurant.name
    iter_address = restaurant.address
    if restaurant.coordinates["latitude"] == None:
        place_ids.append(None)
    else:
        iter_lat = restaurant.coordinates["latitude"]
        iter_long = restaurant.coordinates["longitude"]
        iter_place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={iter_name} {iter_address}&inputtype=textquery&locationbias=circle:50@{iter_lat},{iter_long}&fields=place_id&key={MY_API_KEY}")
        iter_place_dict = json.loads(iter_place_request.text)

        if iter_place_dict["candidates"] == []:
            place_ids.append(None)
        else:
            place_ids.append(iter_place_dict["candidates"][0]["place_id"])


In [89]:
#Now request for each restaurant with required fields
restaurants_google_API=[]
for id in place_ids:
    if id == None:
        restaurants_google_API.append(None)
    else:
        request=requests.get(f'https://maps.googleapis.com/maps/api/place/details/json?place_id={id}&fields=name%2Cformatted_address%2Cgeometry%2Crating%2Cformatted_phone_number%2Cuser_ratings_total&key={MY_API_KEY}') # phone number 
        restaurants_google_API.append(json.loads(request.text))


In [13]:
#It turned out that this function is overcomplicated, simplified version follows

def Find_place(links, API_KEY):
    '''
    This function will:
    1) go through list of links and creates an instance of class Restaurant for each link
    2) for each of the telephones numbers of this instance, searches for a place by this number on Google API
    3) compares tel. numbers of all candidates with all numbers from zlatestranky
    4) if any match is found, it saves the detail of the place, if no match found, saves None
    '''
    places = []
    for link in links:
        restaurant=Restaurant(link)
        result = None 
        #iter_name = restaurant.name
        #iter_address = restaurant.address
        
        iter_phones = [] #this list contains all restaurant's phones (in form "+420 XXX XXX XXX")
        for tel in restaurant.phones:
            iter_phones.append(restaurant.phones[tel])

        if restaurant.coordinates["latitude"] == None:
            places.append(result)
        else:
            for phone in iter_phones: #this forloop should go through phones of resaurant, the search is done repeatedly based on each number                              
                counter = 0
                candidate = 0
                condition = True
                while condition: #this while loop will ensure the cycling through all search candidates
                    
                    iter_lat = restaurant.coordinates["latitude"]
                    iter_long = restaurant.coordinates["longitude"]
                    iter_place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={phone}&inputtype=textquery&locationbias=circle:50@{iter_lat},{iter_long}&fields=place_id&key={API_KEY}")
                    iter_place_dict = json.loads(iter_place_request.text)
                    
                    if iter_place_dict["candidates"] == []:
                        condition = False
                    else:
                        iter_id = iter_place_dict["candidates"][candidate]["place_id"]
                        iter_detail_request = requests.get(f"https://maps.googleapis.com/maps/api/place/details/json?place_id={iter_id}&fields=name%2Cformatted_address%2Cgeometry%2Crating%2Cformatted_phone_number%2Cuser_ratings_total&key={API_KEY}")
                        iter_detail_dict = json.loads(iter_detail_request.text)

                        #try: 
                        #    found_number = iter_detail_dict['result']['formatted_phone_number'] 
                        #except KeyError:
                        #    continue

                        if 'formatted_phone_number' in iter_detail_dict['result']:
                            found_number = iter_detail_dict['result']['formatted_phone_number']        
                            for zs_phone in iter_phones:
                                if len(zs_phone) != 16:
                                    condition = False #non-standard phone number
                                else:
                                    zs_phone_short = zs_phone[4:] #geting rid of '+420 ' part

                                    if zs_phone_short == found_number: #<-- the error might be here?
                                        condition = False
                                        
                                        #saving the result
                                        result = {'ZS_name':restaurant.name, 'name':iter_detail_dict['result']['name'], 'formatted_address':iter_detail_dict['result']['formatted_address'], 'geometry':iter_detail_dict['result']['geometry'] , 'rating':iter_detail_dict['result']['rating'] , 'user_ratings_total':iter_detail_dict['result']['user_ratings_total'] , 'formatted_phone_number':iter_detail_dict['result']['formatted_phone_number']}                                
                                        break
                                    else: 
                                        condition = True
                        else:
                            condition = True
                            
                        candidate += 1

                        if candidate >= len(iter_place_dict["candidates"]):
                            condition = False
                


                    if type(result) == dict:
                        condition = False

                    counter += 1 #this is here in order not to run it forever
                    if counter > 30:
                        condition = False

            places.append(result)

    return places

In [16]:
#Find_place in action
links_sample = links.links[:10]
places_test = Find_place(links_sample, MY_API_KEY)



In [17]:
places_test

[None, None, None, None, None, None, None, None, None, None]

In [101]:
def find_first_candidate(name, phone, coordinates, API_KEY):
    '''
    This function will:
    1) take dictionary of phones of restaurant and takes the first one and dict of coordinates and saves them
    2) uses Find place to get first candidates ID
    3) use this ID to get Details
    4) return details as a dictionary 
    
    The function expects argument phone to be in form {'hlavní telefon': '+420 222 311 221', ...}
    '''
    phone_list = []
    for i in phone:
        phone_list.append(phone[i])
    
    main_phone = phone_list[0] #extracting the main phone from dict

    if coordinates == None:
        result = {'ZS_name':name, 'name':None, 'formatted_address':None, 'location':None, 'rating':None, 'user_ratings_total':None, 'formatted_phone_number':None}
    else:
        latitude = coordinates['latitude']
        longitude = coordinates['longitude']

    place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={main_phone}&inputtype=textquery&locationbias=circle:50@{latitude},{longitude}&fields=place_id&key={API_KEY}")
    place_dict = json.loads(place_request.text)

    if place_dict["candidates"] == []:
        result = {'ZS_name':name, 'name':None, 'formatted_address':None, 'location':None, 'rating':None, 'user_ratings_total':None, 'formatted_phone_number':None}
    else:
        iter_id = place_dict["candidates"][0]["place_id"]
        detail_request = requests.get(f"https://maps.googleapis.com/maps/api/place/details/json?place_id={iter_id}&fields=name%2Cformatted_address%2Cgeometry%2Crating%2Cformatted_phone_number%2Cuser_ratings_total&key={API_KEY}")
        detail_dict = json.loads(detail_request.text)
        
        #saving the details
        result = {'ZS_name':name}

        try:
            result['name'] = detail_dict['result']['name']
        except KeyError:
            result['name'] = None

        try:
            result['formatted_address'] = detail_dict['result']['formatted_address']
        except KeyError:
            result['formatted_address'] = None

        try:
            result['location'] = detail_dict['result']['geometry']['location']
        except KeyError:
            result['location'] = None

        try:
            result['rating'] = detail_dict['result']['rating']
        except KeyError:
            result['rating'] = None

        try:
            result['user_ratings_total'] = detail_dict['result']['user_ratings_total']
        except KeyError:
            result['user_ratings_total'] = None

        try:
            result['formatted_phone_number'] = detail_dict['result']['formatted_phone_number']
        except KeyError:
            result['formatted_phone_number'] = None            
################
#        try:
#            result = {'ZS_name':name, 'name':detail_dict['result']['name'], 'formatted_address':detail_dict['result']['formatted_address'], 'geometry':detail_dict['result']['geometry'] , 'rating':detail_dict['result']['rating'] , 'user_ratings_total':detail_dict['result']['user_ratings_total'] , 'formatted_phone_number':detail_dict['result']['formatted_phone_number']}
#        except KeyError:
#            result = None
    return result

    


In [102]:
#Now to try it on the data frame
zs_restaurants=pd.read_csv('restaurants_zlatestranky.csv')

In [103]:
list_of_results = []
for ind in zs_restaurants[:50].index:
    if zs_restaurants["name"][ind] == "Céleste":
        celeste_exception = {'ZS_name':"Céleste", 'name':None, 'formatted_address':None, 'location':None, 'rating':None, 'user_ratings_total':None, 'formatted_phone_number':None}
        list_of_results.append(celeste_exception)
    else:
        iter_name = zs_restaurants["name"][ind]
        iter_phones = ast.literal_eval(zs_restaurants["phones"][ind])
        iter_coordinates = ast.literal_eval(zs_restaurants["coordinates"][ind])

        iter_result = find_first_candidate(iter_name, iter_phones, iter_coordinates, MY_API_KEY)

        list_of_results.append(iter_result)


In [104]:
list_of_results

[{'ZS_name': 'Repre restaurant',
  'name': 'Repre Restaurant',
  'formatted_address': '857, Nekázanka 4, 110 00 Praha 1-Nové Město, Czechia',
  'location': {'lat': 50.0854596, 'lng': 14.4273116},
  'rating': 4,
  'user_ratings_total': 788,
  'formatted_phone_number': '222 211 451'},
 {'ZS_name': 'Restaurace Nad Ledem',
  'name': 'Restaurace Nad Ledem',
  'formatted_address': '10b, Vladivostocká 1460, Vršovice, 100 00 Praha-Praha 10, Czechia',
  'location': {'lat': 50.0689274, 'lng': 14.4791565},
  'rating': 4.4,
  'user_ratings_total': 369,
  'formatted_phone_number': '728 464 824'},
 {'ZS_name': 'SRI LANKAN CURRY HOUSE',
  'name': None,
  'formatted_address': None,
  'location': None,
  'rating': None,
  'user_ratings_total': None,
  'formatted_phone_number': None},
 {'ZS_name': 'Restaurace HOOTERS Vodičkova',
  'name': 'Na Zdraví Ventures, a.s.',
  'formatted_address': '18, Štěrková 1003, 153 00 Praha 16, Czechia',
  'location': {'lat': 49.9901585, 'lng': 14.3661303},
  'rating': Non

In [105]:
#Saving the dataset
#test_df=pd.DataFrame(list_of_results)
#test_df.to_csv('restaurants_Places_API.csv')

In [75]:
#######################
#Filtering the data
#Empty list
restaurants_filtered=[]
#Keys
keys_new_dict=['name','address','opening_hours','place_id','price_level','rating','no_of_reviews','location']
keys_old_dict=['name','formatted_address','opening_hours','place_id','price_level','rating','user_ratings_total','geometry']
#Looping through restaurants to filter the data
for restaurant in restaurants:
    dict={}
    for key_new, key_old in zip(keys_new_dict, keys_old_dict):
        if key_new=='location': #Location is a dict within adict
            try:
                dict[key_new]=restaurant[key_old][key_new]
            except KeyError:
                dict[key_new]=None
        else: #Rest of the keys
            try:
                dict[key_new]=restaurant[key_old]
            except KeyError:
                dict[key_new]=None
    restaurants_filtered.append(dict)

In [78]:
#Saving the list for future use
with open('restaurants_google_maps_places_API.pkl', 'wb') as data_file:
    pickle.dump(restaurants_filtered, data_file)

In [7]:
#Code to read the file
with open('restaurants_google_maps_places_API.pkl', 'rb') as data_file:
    restaurants_filtered=pickle.load(data_file)

In [10]:
restaurants_filtered

[{'name': 'Restaurant Zvonice',
  'address': 'Jindřišská 33, 110 00 věž-Nové Město, Czechia',
  'place_id': 'ChIJmbWmIO2UC0cR_Gua9yvxWaw',
  'price_level': 3,
  'rating': 4.5,
  'no_of_reviews': 673,
  'location': (50.0851891, 14.4301079)},
 {'name': 'La Finestra in Cucina',
  'address': 'Platnéřská 90/13, 110 00 Staré Město, Czechia',
  'place_id': 'ChIJGxmor-iUC0cRdTmViA87nzs',
  'price_level': 3,
  'rating': 4.6,
  'no_of_reviews': 988,
  'location': (50.08735129999999, 14.4162553)},
 {'name': 'PAPRIKA - Mediterranean Kitchen & Bar',
  'address': 'Rumunská 8/16, 120 00 Vinohrady, Czechia',
  'place_id': 'ChIJvb0ESYmUC0cRmA9ecAWTok8',
  'price_level': 1,
  'rating': 4.7,
  'no_of_reviews': 1260,
  'location': (50.07383249999999, 14.4321147)},
 {'name': 'Bastion Prague Restaurant',
  'address': 'Horská 1751, 128 00 Nové Město, Czechia',
  'place_id': 'ChIJd7kyj2GUC0cRK8h-gH3M1aQ',
  'price_level': 3,
  'rating': 4.1,
  'no_of_reviews': 233,
  'location': (50.06786229999999, 14.4269231

In [9]:
#Adjusting the list before transforming it into a pandas DataFrame
#Putting the coordinates into a tuple
for restaurant in restaurants_filtered:
    restaurant['location']=tuple([restaurant['location']['lat'],restaurant['location']['lng']])
#Removing opening hours since they do not seem to be informative and we need to decrease the file size
for restaurant in restaurants_filtered:
    restaurant.pop('opening_hours')

In [11]:
#Creating a pandas DataFrame
df=pd.DataFrame(restaurants_filtered)

In [12]:
#Writing into a csv file
df.to_csv('restaurants_google_maps_places_API.csv')

In [14]:
#Trying to read the file
df2=pd.read_csv('restaurants_google_maps_places_API.csv')